In [ ]:
# standard imports
import pandas as pd
import numpy as np
from pathlib import Path
import pickle
from tqdm.auto import tqdm
import time
from datetime import timedelta
from collections import Counter, defaultdict
# import polars as pl

# import defined file paths 
from config import *

# data visualisation
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

https://www.kaggle.com/code/edwardcrookenden/otto-getting-started-eda-baseline

In [ ]:
# train = pd.read_parquet(OPT_TRAIN_PATH)
# test = pd.read_parquet(OPT_TEST_PATH)

In [ ]:
with open(TRAIN_PATH, 'r') as f:
    print(f'{len(f.readlines()):,} lines in the training data')

In [ ]:
with open(TEST_PATH, 'r') as f:
    print(f'{len(f.readlines()):,} lines in the training data')

In [ ]:
sample_size = 200000

chunks = pd.read_json(TRAIN_PATH, lines=True, chunksize=sample_size)

for c in chunks:
    train_sample = c.set_index('session')
    break

In [ ]:
train_sample.head()

In [ ]:
first_session = train_sample.iloc[0].item()
print(f'The first session had {len(first_session)} actions')
# Time of session
time_elapsed = first_session[-1]["ts"] - first_session[0]["ts"]
# The timestamp is in milliseconds since 00:00:00 UTC on 1 January 1970
print(f'The first session elapsed: {str(timedelta(milliseconds=time_elapsed))}')

action_counts = defaultdict(int)
for action in first_session:
    a_type = action['type']
    action_counts[a_type] = action_counts[a_type] + 1
    
print(f'The first session had this frequency of actions: {action_counts}')


In [ ]:
train_sample.iloc[0].item()

In [ ]:
dict(Counter(action['aid'] for action in train_sample.iloc[0].item()))

In [ ]:
action_type_count_list, aid_count_list, session_duration_list, session_action_count_list = ([] for _ in range(4))
# frequency of actions of each type (click, cart, order)
total_action_type_counts = Counter()
# frequency of article ids
total_aid_counts = Counter()
for session, actions in tqdm(train_sample.itertuples(), total=len(train_sample)):
    # count the frequency of each action type 
    action_type_count = Counter(action['type'] for action in actions)
    # count the frequency of each article id
    aid_count = Counter((action['aid'] for action in actions))
    
    # add action and aid counts from this session to the total
    total_action_type_counts += action_type_count
    total_aid_counts += aid_count
    
    # record the session duration
    session_duration = actions[-1]['ts'] - actions[0]['ts']
    
    # append to lists
    action_type_count_list.append(dict(action_type_count))
    aid_count_list.append(dict(aid_count))
    session_duration_list.append(session_duration)
    session_action_count_list.append(len(actions)) # count of actions in this session is the length of the actions array

train_sample['action_type_counts'] = action_type_count_list
train_sample['aid_counts'] = aid_count_list
train_sample['duration'] = session_duration_list
train_sample['action_count'] = session_action_count_list